In [18]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from prep_bert import BertEncoder, build_dataloaders
from fine_tune_bert import fine_tune_bert
import pickle
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
with open('data.pickle', 'rb') as handle:
    df = pickle.load(handle)

In [20]:
df.head()

,url,class_label,class_id,text
0,https://www.nytimes.com/2020/08/17/world/afric...,Incident Report,0,Advertisement Supported by The extremist grou...
1,https://www.bbc.com/news/world-europe-54500555,Incident Report,0,Belarusian riot police have used water cannon...
2,https://www.aljazeera.com/news/2020/10/12/nago...,Incident Report,0,Russian foreign minister calls on Armenia and...
3,https://counteriedreport.com/roadside-bomb-bla...,Incident Report,0,Al-Shabaab ended 2019 with a truck-borne impro...
4,https://counteriedreport.com/al-shabaabs-impro...,Incident Report,0,Al-Shabaab ended 2019 with a truck-borne impro...


In [21]:
texts = df['text'].tolist()
label = df['class_id'].apply(lambda x: int(x))


In [22]:
dataset = BertEncoder(tokenizer=BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False), input_data=texts)

In [23]:
data = dataset.tokenize(max_len=510)

In [24]:
input_ids, attention_masks = data
label = torch.Tensor(label.to_list()).long()

In [25]:
train_dataloader, val_dataloader = build_dataloaders(
    input_ids=input_ids,
    attention_masks=attention_masks,
    labels=label,
    batch_size=(16, 4),
    train_ratio=0.8
)

In [26]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased',
    output_attentions=False,
    output_hidden_states=False,
    num_labels = 4
)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
if torch.cuda.is_available():
    device = torch.device('cuda')
else: device = torch.device('cpu')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [27]:
trained_model, stats = fine_tune_bert(
    train_dataloader=train_dataloader, 
    valid_dataloader=val_dataloader, 
    model=model,
    optimizer=optimizer,
    save_model_path='model/trained_model.pt',
    save_stats_dict_path='logs/statistics.json',
    device = device
)

KeyboardInterrupt: 